In [ ]:
main.py
from src import data_preprocessing, feature_engineering, model_training, model_validation, save_model, deployment_plan

def main():
    # Step 1: Data Preprocessing
    data = data_preprocessing.load_data('data/AnomaData.xlsx')
    data = data_preprocessing.clean_data(data)

    # Step 2: Feature Engineering
    X, y = feature_engineering.engineer_features(data)

    # Step 3: Train/Test Split
    X_train, X_test, y_train, y_test = model_training.split_data(X, y)

    # Step 4: Model Training
    rf_model, xgb_model = model_training.train_models(X_train, y_train)

    # Step 5: Model Validation
    best_model, best_params = model_validation.validate_model(rf_model, xgb_model, X_train, y_train, X_test, y_test)

    # Step 6: Save Model
    save_model.save_model(best_model, 'models/anomaly_detection_model.pkl')

    # Step 7: Deployment Plan
    deployment_plan.create_deployment_plan()

if __name__ == "__main__":
    main()

src/data_preprocessing.py
import pandas as pd

def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

def clean_data(data):
    # Handle missing values by filling them with the mean
    data.fillna(data.mean(), inplace=True)

    # Outlier detection and treatment (using IQR method)
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    data = data.apply(lambda x: x.clip(lower=x.quantile(0.05), upper=x.quantile(0.95)))
    
    return data

src/feature_engineering.py
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif

def engineer_features(data):
    # Convert date column to datetime format if exists
    if 'date' in data.columns:
        data['date'] = pd.to_datetime(data['date'])
        data['day_of_week'] = data['date'].dt.dayofweek
        data['hour'] = data['date'].dt.hour
        data = data.drop(columns=['date'])

    X = data.drop(columns=['y'])
    y = data['y']

    # Feature Selection
    selector = SelectKBest(f_classif, k=10)
    X_new = selector.fit_transform(X, y)
    selected_features = X.columns[selector.get_support()]
    
    print(f"Selected Features: {selected_features}")
    
    return X_new, y

src/feature_engineering.py
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif

def engineer_features(data):
    # Convert date column to datetime format if exists
    if 'date' in data.columns:
        data['date'] = pd.to_datetime(data['date'])
        data['day_of_week'] = data['date'].dt.dayofweek
        data['hour'] = data['date'].dt.hour
        data = data.drop(columns=['date'])

    X = data.drop(columns=['y'])
    y = data['y']

    # Feature Selection
    selector = SelectKBest(f_classif, k=10)
    X_new = selector.fit_transform(X, y)
    selected_features = X.columns[selector.get_support()]
    
    print(f"Selected Features: {selected_features}")
    
    return X_new, y

src/model_training.py
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    return X_train, X_test, y_train, y_test

def train_models(X_train, y_train):
    # Train RandomForest
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)

    # Train XGBoost
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)

    return rf_model, xgb_model

src/model_validation.py
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

def validate_model(rf_model, xgb_model, X_train, y_train, X_test, y_test):
    # Evaluate RandomForest
    y_pred_rf = rf_model.predict(X_test)
    rf_accuracy = accuracy_score(y_test, y_pred_rf)
    rf_classification_report = classification_report(y_test, y_pred_rf)
    print(f"RandomForest Accuracy: {rf_accuracy}")
    print(rf_classification_report)

    # Hyperparameter Tuning (example with GridSearchCV for RandomForest)
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
    }
    grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Best parameters and model
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    print(f"Best Parameters: {best_params}")

    return best_model, best_params

src/save_model.py
import joblib

def save_model(model, file_path):
    joblib.dump(model, file_path)
    print(f"Model saved to {file_path}")

src/deployment_plan.py
def create_deployment_plan():
    deployment_plan = """
    1. Save the trained model (done above).
    2. Develop an API using Flask or FastAPI to serve predictions.
    3. Containerize the application using Docker.
    4. Deploy on a cloud platform (AWS, Azure, GCP).
    5. Set up monitoring and logging for the deployed model.
    """
    print(deployment_plan)

notebooks/exploratory_data_analysis.ipynb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
file_path = '../data/AnomaData.xlsx'
data = pd.read_excel(file_path)

# Inspect data
print(data.head())

# Check for missing values
missing_values = data.isnull().sum()
print(missing_values)

# Check data types
data_types = data.dtypes
print(data_types)

# Summary statistics
summary_stats = data.describe()
print(summary_stats)

# Histograms for each feature
data.hist(bins=50, figsize=(20,15))
plt.show()

# Correlation matrix
corr_matrix = data.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()
